# Chassis service

Import libraries that have been installed before

In [2]:
import sklearn
import modzymodel

### Train the model

This will train a sklearn model and it will be saved as a joblib file inside the `model` directory.

The goal for Chassis service is to create an image that exposes this model.

In [3]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from joblib import dump, load

digits = datasets.load_digits()

data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

dump(clf, './model/model.joblib')

['./model/model.joblib']

### Train the model

Transform the model into MLFlow format.

In [4]:
import joblib
clf = joblib.load('./model/model.joblib')

import mlflow.sklearn
mlflow.sklearn.save_model(clf, './model/MLFlowModel')

MlflowException: Path './model/MLFlowModel' already exists

We check that the model has been correctly saved inside the `model` directory.

In [5]:
!ls ./model/MLFlowModel

conda.yaml  MLmodel  model.pkl


### Define the values needed

The fields `name`, `version` and `api_key` are used when uploading the image to Modzy platform.

Since now we are just creating and downloading the docker image, the only fields that Chassis service actually needs are:

* `model_name`: name for the model inside the image
* `model_path`: directory that contains our model file

In [6]:
api_key = 'XXXX'

image_data = {
    'name': 'docker-registry:5000/mlflow-digits-containerized',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': './model/MLFlowModel',
}

### Forward ports to access service and registry

This assumes that you are running these commands on your own terminal to redirect the service (port 5000) and the registry (port 5001) to localhost.

In [6]:
! # kubectl port-forward service/chassis 5000:5000
! # kubectl port-forward service/docker-registry 5001:5000

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: the name of the service that runs Chassis

In [7]:
res = modzymodel.publish(
    api_key=api_key,
    module=mlflow,
    image_data=image_data,
    deploy=True,
    image_type=modzymodel.Constants.IMAGE_GREY,
    base_url='http://localhost:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Publishing container... Ok!
Job ID: chassis-builder-job-e7c8e792-e16d-439f-9621-3ee5f887604a


After the request is made, Chassis launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

In [8]:
modzymodel.get_job_status(job_id)

Not finished yet. Try again later


Now, we should be able to see the created image listed in the registry. This means that the service has correctly created the image and uploaded it.

In [9]:
!wget -qO- http://localhost:5001/v2/_catalog

{"repositories":["mlflow-digits-containerized"]}


### Pull the docker image

Now that the job has finished, we can pull and load the docker image that has been generated.

In [10]:
!docker pull localhost:5001/mlflow-digits-containerized

Using default tag: latest
latest: Pulling from mlflow-digits-containerized

2152171a: Already exists 
043ff07a: Pulling fs layer 
01523801: Pulling fs layer 
db539500: Pulling fs layer 
16605b55: Pulling fs layer 
903809c0: Pull complete 498MB/5.498MBBADigest: sha256:3db68a482375ed0a5c46894825a9030bba0537f05ea6768b073594b9fe6ecaa9
Status: Downloaded newer image for localhost:5001/mlflow-digits-containerized:latest
localhost:5001/mlflow-digits-containerized:latest


In [11]:
!docker images localhost:5001/mlflow-digits-containerized

REPOSITORY                                   TAG       IMAGE ID       CREATED          SIZE
localhost:5001/mlflow-digits-containerized   latest    af270737e355   21 seconds ago   691MB
